<h1>Data generation for Energy-Specific study</h1>

We want to generate data for all assemblies to determine the relationship between the NN analysis accuracy and the assembly size.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import copy

import import_ipynb
from NuclearAssemblies import *

import pandas as pd

<h3>Selection of energy</h3>

As we discuss in the Data Generation section (sec 2.5) in the dissertation, we are going to focus on the fluence for a specific energy, in particular on peaks of fluence at the lower energies region of the spectra.

In [ ]:
energies = SpectraDict['4JR_tmirod_uo2_spectrum.txt'].data[0] # The smallest energy range
MaxFluenceEnergies = energies[[np.argmax(SpectraDict['4JR_atm106_spectrum.txt'].data[1]),
                              np.argmax(SpectraDict['4JR_moxrod_spectrum2.txt'].data[1]),
                              np.argmax(SpectraDict['4JR_tmirod_uo2_spectrum.txt'].data[1])]]
#print(MaxFluenceEnergies) # All equal 

Energy = MaxFluenceEnergies[0]
print(Energy*1000, "keV.")

Which we notice is the $\text{K}_\alpha$ XRF emission for uranium. This fact doesn't really mean anything, but it's a nice titbit.

Let us generate data focusing only on this energy for now.

In [ ]:
idx=[np.where(energies==0.099)[0][0]]
idx

<h3>Radii selection</h3>

For the 7 by 7 assembly we chose a radius of 0.065 for the detector. Remember that the assembly gap size was defined at 0.0125. Let us call $H$ the distance from the center of the assembly to its side (not the corner), assuming puntual pins.

In [ ]:
ass7=assembly('Assembly7TestData.txt')
gap=ass7.gap
overhead=0.065-gap*3*np.sqrt(2)
print(overhead)

Let us generate the radii for the rest of assemblies with $s=$ 3, 5, 7, 9, 11, 13, 15, 17.

In [ ]:
sizes=np.array([11,13,15])
radii=np.array([overhead+gap*(s-1)/2*np.sqrt(2)for s in sizes])

radii2=radii*(0.1/0.065)

<h2>Data generation</h2>

In [ ]:
###------###    WHOLLY NON-CONVEX ASSEMBLIES    ###------###
print('Data generation for NON-CONVEX assemblies innitiated.\n')

for i in range(len(sizes)):

    print('Data generation for size ',sizes[i],'.')
    
    k=sizes[i]**2-1
    mmin=0
    mvalues=np.arange(mmin+1,k+1)
    lsetsm=200
    assname='Assembly'+str(sizes[i])+'TestData.txt'
    baseDet=detector(assembly(assname),(radii[i],0),compSpectra=False)

    det=copy.deepcopy(baseDet)
    det.pop_pinRand(mmin)                                      # (2) 65 us
    
    m0entry=np.delete(det.gen_dataentry()[:,idx],0,axis=0)     # (3) 100 ms 
    
    xdataset=[m0entry]                                         # (6) 36 ms
    ydataset=[np.sum(det.assembly.pinslots,axis=2)]
    for l in np.arange(1,lsetsm):
        xdataset=np.append(xdataset,[m0entry],axis=0)
        ydataset=np.append(ydataset,[np.sum(det.assembly.pinslots,axis=2)],axis=0)
    
    for m in mvalues:
        for l in range(lsetsm):
            det=copy.deepcopy(baseDet)
            det.pop_pinRand(m)
            xdataset=np.append(xdataset,[np.delete(det.gen_dataentry()[:,idx],0,axis=0)],axis=0)
            ydataset=np.append(ydataset,[np.sum(det.assembly.pinslots,axis=2)],axis=0)
            if (l+1)%(lsetsm/10)==0 or l==0:
                output='m = '+str(m)+' /'+str(k)+' : '
                for per in range(10):
                    if l/(lsetsm/10)>per: add='.' 
                    else: add=' '
                    output=output+add
                print(output,end='\r')
    print('                          \rDone')

    dataframe=pd.DataFrame({'AssemblyPinslots':list(ydataset.astype('int16')),
                            'SpectraCylinder':list(xdataset.astype('float32'))}) 
            # We need to pass array arguments as lists
    dataname='FluDat_ESS'+str(sizes[i])+'.pickle'
    dataframe.to_pickle(dataname)
    
    print(dataname,' saved to disk.\n')
    
    del(xdataset,ydataset,dataframe)

In [ ]:
###------###    DOUBLE RADIUS NON-CONVEX    ###------###
print('Data generation for DOUBLE RADIUS NON-CONVEX assemblies innitiated.\n')

for i in range(len(sizes)):

    print('Data generation for size ',sizes[i],'.')
    
    k=sizes[i]**2-1
    mmin=0
    mvalues=np.arange(mmin+1,k+1)
    lsetsm=200
    assname='Assembly'+str(sizes[i])+'TestData.txt'
    baseDet=detector(assembly(assname),(radii[i],0),compSpectra=False)

    det=copy.deepcopy(baseDet)
    det.pop_pinRand(mmin)                                      # (2) 65 us
    
    m0entry1=np.delete(det.gen_dataentry()[:,idx],0,axis=0)    # (3) 100 ms
    det.set_radius(radii2[i])
    m0entry2=np.delete(det.gen_dataentry()[:,idx],0,axis=0)    # (3) 100 ms
    
    xdataset=[[m0entry1,m0entry2]]                             # (6) 36 ms
    ydataset=[np.sum(det.assembly.pinslots,axis=2)]
    for l in np.arange(1,lsetsm):
        xdataset=np.append(xdataset,[[m0entry1,m0entry2]],axis=0)
        ydataset=np.append(ydataset,[np.sum(det.assembly.pinslots,axis=2)],axis=0)
    
    for m in mvalues:
        for l in range(lsetsm):
            det=copy.deepcopy(baseDet)
            det.pop_pinRand(m)
            
            entry1=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
            det.set_radius(radii2[i])
            entry2=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
            
            xdataset=np.append(xdataset,[[entry1,entry2]],axis=0)
            ydataset=np.append(ydataset,[np.sum(det.assembly.pinslots,axis=2)],axis=0)
            if (l+1)%(lsetsm/10)==0 or l==0:
                output='m = '+str(m)+' /'+str(k)+' : '
                for per in range(10):
                    if l/(lsetsm/10)>per: add='.' 
                    else: add=' '
                    output=output+add
                print(output,end='\r')
    print('                          \rDone')

    dataframe=pd.DataFrame({'AssemblyPinslots':list(ydataset.astype('int16')),
                            'SpectraCylinder':list(xdataset.astype('float32'))}) 
            # We need to pass array arguments as lists
    dataname='FluDatDoubleR_ESS'+str(sizes[i])+'.pickle'
    dataframe.to_pickle(dataname)
    
    print(dataname,' saved to disk.\n')
    
    del(xdataset,ydataset,dataframe)

In [ ]:
###------###    CONVEX ASSEMBLIES    ###------###
print('Data generation for CONVEX assemblies innitiated.\n')

for i in range(len(sizes)):

    print('Data generation for size ',sizes[i],'.')
    
    k=sizes[i]**2-1
    mmin=0
    mvalues=np.arange(mmin,k+1)
    lsetsm=200
    assname='Convex_Assembly'+str(sizes[i])+'TestData.txt' #
    baseDet=detector(assembly(assname),(radii[i],0),compSpectra=False)
    
        #initiating data
    xdataset=[np.delete(baseDet.gen_dataentry()[:,idx],0,axis=0)]
    ydataset=[np.round(np.sum(baseDet.assembly.pinslots,axis=2))]
    
    for m in mvalues:
        for l in range(lsetsm):
                # We reset the detectors and re-asign spectra to the pins
            det=copy.deepcopy(baseDet)
            det.assembly._asign_pinslots()
            
            det.pop_pinRand(m)                                 
            xdataset=np.append(xdataset,[np.delete(det.gen_dataentry()[:,idx],0,axis=0)],axis=0)
            ydataset=np.append(ydataset,[np.round(np.sum(det.assembly.pinslots,axis=2))],axis=0)
                #np.round is VERY important to avoid problems when converting to int16
            
            if (l+1)%(lsetsm/10)==0 or l==0:
                output='m = '+str(m)+' /'+str(k)+' : '
                for per in range(10):
                    if l/(lsetsm/10)>per: add='.' 
                    else: add=' '
                    output=output+add
                print(output,end='\r')
    print('                          \rDone')
    
            # We delete the initiating data
    xdataset=np.delete(xdataset,0,axis=0)
    ydataset=np.delete(ydataset,0,axis=0)

    dataframe=pd.DataFrame({'AssemblySumPinslots':list(ydataset.astype('int16')),
                            'SpectraCylinder':list(xdataset.astype('float32'))}) # 
            # We need to pass array arguments as lists
    dataname='Convex_FluDat_ESS'+str(sizes[i])+'.pickle' #
    dataframe.to_pickle('./Pickle/'+dataname)
    
    print(dataname,' saved to disk.\n')
    
    del(xdataset,ydataset,dataframe)

In [ ]:
###------###    DOUBLE RADIUS CONVEX ASSEMBLIES    ###------###
print('Data generation for DOUBLE RADIUS CONVEX assemblies innitiated.\n')

for i in range(len(sizes)):

    print('Data generation for size ',sizes[i],'.')
    
    k=sizes[i]**2-1
    mmin=0
    mvalues=np.arange(mmin,k+1)
    lsetsm=200
    assname='Convex_Assembly'+str(sizes[i])+'TestData.txt' #
    baseDet=detector(assembly(assname),(radii[i],0),compSpectra=False)
    
        #initiating data
    det=copy.deepcopy(baseDet)
    m0entry1=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
    det.set_radius(radii2[i])
    m0entry2=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
    
    xdataset=[[m0entry1,m0entry2]]
    ydataset=[np.round(np.sum(baseDet.assembly.pinslots,axis=2))]
    
    for m in mvalues:
        for l in range(lsetsm):
                # We reset the detectors and re-asign spectra to the pins
            det=copy.deepcopy(baseDet)
            det.assembly._asign_pinslots()
            det.pop_pinRand(m)      
            
            entry1=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
            det.set_radius(radii2[i])
            entry2=np.delete(det.gen_dataentry()[:,idx],0,axis=0)
            
            xdataset=np.append(xdataset,[[entry1,entry2]],axis=0)
            ydataset=np.append(ydataset,[np.round(np.sum(det.assembly.pinslots,axis=2))],axis=0)
                #np.round is VERY important to avoid problems when converting to int16
            
            if (l+1)%(lsetsm/10)==0 or l==0:
                output='m = '+str(m)+' /'+str(k)+' : '
                for per in range(10):
                    if l/(lsetsm/10)>per: add='.' 
                    else: add=' '
                    output=output+add
                print(output,end='\r')
    print('                          \rDone')
    
            # We delete the initiating data
    xdataset=np.delete(xdataset,0,axis=0)
    ydataset=np.delete(ydataset,0,axis=0)

    dataframe=pd.DataFrame({'AssemblySumPinslots':list(ydataset.astype('int16')),
                            'SpectraCylinder':list(xdataset.astype('float32'))}) # 
            # We need to pass array arguments as lists
    dataname='Convex_FluDatDoubleR_ESS'+str(sizes[i])+'.pickle' #
    dataframe.to_pickle('./Pickle/'+dataname)
    
    print(dataname,' saved to disk.\n')
    
    del(xdataset,ydataset,dataframe)